# AWS Marketplace Product Usage Demonstration - Online-purchasing-intention-prediction Model Package

## 1. Set up the environment

In [1]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from sagemaker.serializers import CSVSerializer
import boto3
import json

role = get_execution_role()

# S3 prefixes
common_prefix = "<your S3 bucket name>"
batch_inference_input_prefix = "purchasing_intention_prediction/input"
batch_inference_output_prefix = "purchasing_intention_prediction/output"
from sagemaker.predictor import Predictor

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## 2. Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our Amazon SageMaker operations.

In [2]:
sagemaker_session = sage.Session()

## 3. Create Model

Now we use the above Model Package to create a model

In [3]:
model_package_arn = '<your sagemaker model package arn>'

In [10]:
model_name = 'online-purchasing-intention-prediction'

content_type = 'application/json'

real_time_inference_instance_type = 'ml.m5.large'
batch_transform_inference_instance_type = 'ml.m5.large'

In [11]:
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session, content_type)

# Create a deployable model from the model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls = predict_wrapper)

## 4. Create-an-endpoint-and-perform-real-time-inference

#### A. Create an endpoint

In [12]:
#Deploy the model
predictor = model.deploy(1,real_time_inference_instance_type,endpoint_name = model_name)

-----!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [ ]:
file_name = 'input/test.json'

#### C. Perform real-time inference

In [14]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    output.txt

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [15]:
with open('output.txt', 'r') as f:
    output = json.load(f)
print(json.dumps(output,indent=1))

0


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [16]:
predictor.delete_endpoint(delete_endpoint_config = True)

## 5. Perform batch inference


In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

### A. Configure the input S3 bucket folder

In [17]:
transform_input_folder = 'input'
transform_input = sagemaker_session.upload_data(transform_input_folder,common_prefix,batch_inference_input_prefix)

### B. Deploy the model

In [18]:
transformer = model.transformer(1,batch_transform_inference_instance_type,output_path="s3://"+common_prefix+"/"+batch_inference_output_prefix+"/")
transformer.transform(transform_input,content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating transform job with name: online-purchasing-intention-prediction-2024-06-05-07-29-24-908


...........................
Starting the inference server with 2 workers.
[2024-06-05 07:33:51 +0000] [9] [INFO] Starting gunicorn 22.0.0
[2024-06-05 07:33:51 +0000] [9] [INFO] Listening at: unix:/tmp/gunicorn.sock (9)
[2024-06-05 07:33:51 +0000] [9] [INFO] Using worker: sync
[2024-06-05 07:33:51 +0000] [12] [INFO] Booting worker with pid: 12
[2024-06-05 07:33:52 +0000] [14] [INFO] Booting worker with pid: 14
Starting the inference server with 2 workers.
[2024-06-05 07:33:51 +0000] [9] [INFO] Starting gunicorn 22.0.0
[2024-06-05 07:33:51 +0000] [9] [INFO] Listening at: unix:/tmp/gunicorn.sock (9)
[2024-06-05 07:33:51 +0000] [9] [INFO] Using worker: sync
[2024-06-05 07:33:51 +0000] [12] [INFO] Booting worker with pid: 12
[2024-06-05 07:33:52 +0000] [14] [INFO] Booting worker with pid: 14
169.254.255.130 - - [05/Jun/2024:07:33:58 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [05/Jun/2024:07:33:58 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-h

### C. Download the file from output S3 bucket folder

In [19]:
import os
s3_conn = boto3.client("s3")
with open('output2.txt','wb') as f:
    s3_conn.download_fileobj(common_prefix,batch_inference_output_prefix+'/'+'test.json.out',f)
    print("Output file loaded from bucket")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Output file loaded from bucket


### D. Visualize data

In [20]:
with open('output2.txt','r') as f:
    output = json.load(f)
print(json.dumps(output,indent=1))

0


### 6. Clean-up


### A. Delete the model

In [21]:
model.delete_model()

INFO:sagemaker:Deleting model with name: online-purchasing-intention-prediction-2024-06-05-07-29-24-106
